In [3]:
import numpy as np
import torch
from torch.autograd import Variable
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision
import torch.nn as nn
from torch.optim import lr_scheduler
from torchvision import datasets, transforms, models
import torch.nn.functional as F
import timeit
%matplotlib inline

In [ ]:
# The model FCNS 32
class fcns32(nn.Module):
    
    
    def __init__(self, n_classes =21, learn_bilinear=False):
        self.learn_bilinear = learn_bilinear
        self.n_classes = n_classes
        
        # Creating the convolutional blocks
        self.conv1block = nn.Sequential(
                            nn.Conv2d(3, 64, 3, padding=1),
                            nn.ReLU(inplace=True),
                            nn.Conv2d(64, 64, 3, padding=1),
                            nn.ReLU(),
                            nn.MaxPool2d(2, stride=2, ceil_mode=True),
                            )
        self.conv2block = nn.Sequential(
                            nn.Conv2d(64, 128, 3, padding=1),
                            nn.ReLU(inplace=True),
                            nn.Conv2d(128, 128, 3, padding=1),
                            nn.ReLU(inplace=True),
                            nn.MaxPool2d(2, stride=2, ceil_mode=True),)
        self.conv3block = nn.Sequential(
                            nn.Conv2d(128, 256, 3, padding=1),
                            nn.ReLU(inplace=True),
                            nn.Conv2d(256,256,3, padding=1),
                            nn.ReLU(inplace=True),
                            nn.Conv2d(256,256,3, padding=1),
                            nn.ReLU(inplace=True),
                            nn.MaxPool2d(2, stride=2, ceil_mode=True),
                            )
        self.conv4block = nn.Sequential(
                            nn.Conv2d(256, 256, 3, padding=1),
                            nn.ReLU(inplace=True),
                            nn.Conv2d(256,512,3, padding=1),
                            nn.ReLU(inplace=True),
                            nn.Conv2d(512,512,3, padding=1),
                            nn.ReLU(inplace=True),
                            nn.MaxPool2d(2, stride=2, ceil_mode=True),
                            )
        self.conv5block = nn.Sequential(
                            nn.Conv2d(512, 512, 3, padding=1),
                            nn.ReLU(inplace=True),
                            nn.Conv2d(512,512,3, padding=1),
                            nn.ReLU(inplace=True),
                            nn.Conv2d(512,512,3, padding=1),
                            nn.ReLU(inplace=True),
                            nn.MaxPool2d(2, stride=2, ceil_mode=True),
                            )
        
        self.classifier = nn.Sequential(
                            nn.Conv2d(512, 4096, 7, padding=1),
                            nn.ReLU(inplace=True),
                            nn.Conv2d(4096, 4096, 1),
                            nn.ReLU(inplace=True),
                            nn.Dropout2d(),
                            nn.Conv2d(4096, n_classes, 1),
                            )
        
    
    # The forward pass
    def forward(self, x):
        conv1 = self.conv1block(x)
        conv2 = self.conv2block(conv1)
        conv3 = self.conv3block(conv2)
        conv4 = self.conv4block(conv3)
        conv5 = self.conv5block(conv4)
        
        output = self.classifier(conv5)
        
        out = F.upsample_bilinear(output, x.size()[2:])
        
        return out
    
    #Using the pretrained network
    def init_vgg16_params(self, vgg16, copy_fc8=True):
        blocks = [self.conv1block,
                 self.conv2block,
                 self.conv3block,
                 self.conv4block,
                 self.conv5block]
        
        ranges = [[0,4], [5,9], [10, 16], [17, 23], [24, 29]]
        features = list(vgg16.features.children())
        print(features)
        
        for i, conv in enumerate(blocks):
            for l1, l2 in zip(features[ranges[i][0]:ranges[i][1]], conv):
                
        